In [ ]:
!pip install -U tensorflow-text

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import tensorflow_text
from google.colab import files
import pandas as pd

from sklearn.svm import SVC, SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as prf

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('drive/MyDrive/data.csv')
text_var = 'text'
class_var = 'mark'

In [ ]:
data.head() 

In [ ]:
df = data[class_var].value_counts(normalize=True) * 100
df.plot.bar(x=class_var)

In [ ]:
data = data[(data[class_var] != -1)]

In [ ]:
marks_new = []
for i, mark in enumerate(data[class_var]):
    if mark == 2:
        marks_new.append(1)
    else:
        marks_new.append(mark)
class_var = 'class_new'
data[class_var] = marks_new
del marks_new

In [ ]:
train_vol = 0.7

In [ ]:
train, test = train_test_split(data, train_size = train_vol, random_state = 99, stratify = data[class_var])
train = train.drop_duplicates(subset=[text_var])

In [ ]:
check = []
train_data = train[text_var].tolist()
for text in test[text_var]:
    if text not in train_data:
        check.append('KEEP')
    else:
        check.append('IGNORE')
test['check'] = check
test = test.loc[test['check'] == 'KEEP']
test = test.drop(['check'], axis=1)
del check

In [ ]:
train_embeddings = []
for text in train_data:
    embedding = embed(str(text))[0]
    train_embeddings.append(embedding)

In [ ]:
test_embeddings = []
for text in test[text_var]:
    embedding = embed(str(text))[0]
    test_embeddings.append(embedding)

In [ ]:
# nbg = GaussianNB()
# nbb = BernoulliNB()
# lg = LogisticRegressionCV(cv=5, multi_class='ovr', max_iter=1200)
svm = SVC(kernel='poly', gamma='scale', probability=True)

In [ ]:
classes = train[class_var].tolist()
classes_to_check = test[class_var].tolist()

In [ ]:
# nbg.fit(train_embeddings,classes)
# nbb.fit(train_embeddings,classes)
# lg.fit(train_embeddings,classes)
svm.fit(train_embeddings,classes)

In [ ]:
# classesNBG = nbg.predict(test_embeddings)
# classesNBB = nbb.predict(test_embeddings)
# classesLG = lg.predict(test_embeddings)
classesSVM = svm.predict(test_embeddings)

In [ ]:
metrics = prfs(classes_to_check, classesSVM, pos_label = 0, average = 'binary')